In [ ]:
! pip install -Uq torch tensorboard sentence-transformers datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
tensorflow 2.17.1 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.18.0 which is incompatible.


In [ ]:
import wandb
wandb.init(mode="disabled")

#### **Create and Prepare embedding dataset**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("./amharic-dataset-binary", split="train")
dataset = dataset.filter(lambda row: row["label"]==1)#.select(range(10000))
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/432 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/132M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/124652 [00:00<?, ? examples/s]

Filter:   0%|          | 0/124652 [00:00<?, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'label'],
    num_rows: 31163
})

In [ ]:
# rename columns
dataset = dataset.rename_column("query", "anchor")
dataset = dataset.rename_column("passage", "positive")

In [ ]:
# Add an id column to the dataset
dataset = dataset.add_column("id", range(len(dataset)))
dataset

Flattening the indices:   0%|          | 0/31163 [00:00<?, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
    num_rows: 31163
})

In [ ]:
# Split dataset into a 10% test set
dataset = dataset.train_test_split(test_size=0.1, seed=7)
dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
        num_rows: 28046
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
        num_rows: 3117
    })
})

In [ ]:
# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Creating json from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

12602505

#### **Create baseline and evaluate pretrained model**

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator, SequentialEvaluator
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets

# load test dataset
test_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])

corpus_dataset

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
    num_rows: 31163
})

In [ ]:
# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
) # Our corpus (cid => document)
queries = dict(
    zip(test_dataset["id"], test_dataset["anchor"])
) # Our queries (qid => question)

In [ ]:
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}
for q_id in queries:
  relevant_docs[q_id] = [q_id]

In [ ]:
matryoshka_dimensions = [512, 384, 256, 128, 64]

matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
  ir_evaluator = InformationRetrievalEvaluator(
      queries=queries,
      corpus=corpus,
      relevant_docs=relevant_docs,
      name=f"dim_{dim}",
      truncate_dim=dim,
      score_functions={"cosine": cos_sim}
  )
  matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

#### **Evaluate BERT Medium Amharic**

In [ ]:
# BGE Small
import torch
from sentence_transformers import SentenceTransformer

model_id = "yosefw/bert-medium-am-embed"

# Load a model
model = SentenceTransformer(
    model_id,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# Evaluate the model
results = evaluator(model)

for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_ndcg@10"
  print(f"{key}: {results[key]}")

dim_512_cosine_ndcg@10: 0.20264107539931023
dim_384_cosine_ndcg@10: 0.18799786773382648
dim_256_cosine_ndcg@10: 0.15167351633481868
dim_128_cosine_ndcg@10: 0.11232552161041276
dim_64_cosine_ndcg@10: 0.06116486667657463


In [ ]:
# print the main score
for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_recall@5"
  print(f"{key}: {results[key]}")

dim_512_cosine_recall@5: 0.2386910490856593
dim_384_cosine_recall@5: 0.22264998395893487
dim_256_cosine_recall@5: 0.17837664420917548
dim_128_cosine_recall@5: 0.13538658966955405
dim_64_cosine_recall@5: 0.07539300609560474


#### **Define loss function with Matryoshka Representation**

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerModelCardData

model_id = "yosefw/bert-medium-am-embed"

model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="BERT Amharic Text Embedding Medium"
    )
)

In [ ]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [512, 384, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

#### **Fine-tune embedding model with** `SentenceTransformersTrainer`

In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")

args = SentenceTransformerTrainingArguments(
    output_dir="bert-medium-amharic-embedding-matryoshka",
    num_train_epochs=4,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=64,
    warmup_ratio=0.1,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    fp16=True,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to=None,
    logging_steps=10,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",
)

In [ ]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=model,
    args=args, # training arguments
    train_dataset=train_dataset.select_columns(
        ["anchor", "positive"]
    ), # training dataset
    loss=train_loss,
    evaluator=evaluator
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 384 Cosine Accuracy@1,Dim 384 Cosine Accuracy@3,Dim 384 Cosine Accuracy@5,Dim 384 Cosine Accuracy@10,Dim 384 Cosine Precision@1,Dim 384 Cosine Precision@3,Dim 384 Cosine Precision@5,Dim 384 Cosine Precision@10,Dim 384 Cosine Recall@1,Dim 384 Cosine Recall@3,Dim 384 Cosine Recall@5,Dim 384 Cosine Recall@10,Dim 384 Cosine Ndcg@10,Dim 384 Cosine Mrr@10,Dim 384 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
1,1.125700,No log,0.536413,0.673725,0.722810,0.783446,0.536413,0.224575,0.144562,0.078345,0.536413,0.673725,0.722810,0.783446,0.657169,0.617110,0.623141,0.540584,0.668271,0.717036,0.785050,0.540584,0.222757,0.143407,0.078505,0.540584,0.668271,0.717036,0.785050,0.657752,0.617648,0.623417,0.523580,0.665704,0.715432,0.774783,0.523580,0.221901,0.143086,0.077478,0.523580,0.665704,0.715432,0.774783,0.647095,0.606471,0.612574,0.506577,0.653513,0.700032,0.758101,0.506577,0.217838,0.140006,0.075810,0.506577,0.653513,0.700032,0.758101,0.630811,0.590291,0.596793,0.454283,0.598332,0.650946,0.720885,0.454283,0.199444,0.130189,0.072089,0.454283,0.598332,0.650946,0.720885,0.583691,0.540317,0.547520,0.583691
2,0.541700,No log,0.563683,0.699711,0.750080,0.808149,0.563683,0.233237,0.150016,0.080815,0.563683,0.699711,0.750080,0.808149,0.683132,0.643427,0.648968,0.559512,0.695861,0.742701,0.806224,0.559512,0.231954,0.148540,0.080622,0.559512,0.695861,0.742701,0.806224,0.680105,0.640082,0.645773,0.555662,0.695220,0.742060,0.796278,0.555662,0.231740,0.148412,0.079628,0.555662,0.695220,0.742060,0.796278,0.674367,0.635532,0.641703,0.544113,0.683029,0.726981,0.789862,0.544113,0.227676,0.145396,0.078986,0.544113,0.683029,0.726981,0.789862,0.663950,0.624077,0.629836,0.501765,0.638114,0.686237,0.754572,0.501765,0.212705,0.137247,0.075457,0.501765,0.638114,0.686237,0.754572,0.624611,0.583533,0.590095,0.624611
3,0.300200,No log,0.581007,0.708053,0.758742,0.817453,0.581007,0.236018,0.151748,0.081745,0.581007,0.708053,0.758742,0.817453,0.695753,0.657246,0.662763,0.576516,0.704203,0.755855,0.814886,0.576516,0.234734,0.151171,0.081489,0.576516,0.704203,0.755855,0.814886,0.691857,0.652964,0.658711,0.571383,0.706128,0.751363,0.805903,0.571383,0.235376,0.150273,0.080590,0.571383,0.706128,0.751363,0.805903,0.686278,0.648238,0.654265,0.557267,0.694578,0.736927,0.793070,0.557267,0.231526,0.147385,0.079307,0.557267,0.694578,0.736927,0.793070,0.673404,0.635294,0.641411,0.516201,0.651909,0.700995,0.763234,0.516201,0.217303,0.140199,0.076323,0.516201,0.651909,0.700995,0.7632

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=876, training_loss=1.2169998289515438, metrics={'train_runtime': 2040.6772, 'train_samples_per_second': 54.974, 'train_steps_per_second': 0.429, 'total_flos': 0.0, 'train_loss': 1.2169998289515438, 'epoch': 3.9840546697038723})

In [ ]:
# save the best model
trainer.save_model()

#### **Evaluate fine-tuned model against baseline**

In [ ]:
from sentence_transformers import SentenceTransformer

fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)

# Evaluate the model
results = evaluator(fine_tuned_model)

# print the main score
for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_ndcg@10"
  print(f"{key}: {results[key]}")

dim_512_cosine_ndcg@10: 0.6955997941650577
dim_384_cosine_ndcg@10: 0.6917635418463226
dim_256_cosine_ndcg@10: 0.6867007553154383
dim_128_cosine_ndcg@10: 0.6733757471582646
dim_64_cosine_ndcg@10: 0.6374396211191251


In [ ]:
# print the main score
for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_recall@5"
  print(f"{key}: {results[key]}")

dim_512_cosine_recall@5: 0.7587423804940648
dim_384_cosine_recall@5: 0.7552133461661854
dim_256_cosine_recall@5: 0.7520051331408405
dim_128_cosine_recall@5: 0.737247353224254
dim_64_cosine_recall@5: 0.7009945460378569


In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 512, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# The sentences to encode
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "በአክሱም ከተማ የሚገኙ ሙስሊም ሴት ተማሪዎች ከሒጃብ መልበስ ጋር በተያያዘ ውዝግብ ከትምህርት ገበታ ውጭ ሆነው እንደሚገኙ የትግራይ እስልምና ጉዳዮች ምክርቤት ስታወቀ። ይህን ለመፍታት ከክልሉ ትምህርት ቢሮ ጋር ንግግር ላይ መሆኑም የክልሉ እስልምና ጉዳዮች ምክርቤት ለዶቼቬለ ገልጿል።",
  "በማዕከላዊ ኢትዮጵያ ክልል ሃድያ ዞን ጊቤ ወረዳ በሚገኙ 12 ቀበሌዎች መሠረታዊ የመንግሥት አገልግሎት መስጫ ተቋማት በሙሉና በከፊል በመዘጋታቸው መቸገራቸውን ነዋሪዎች አመለከቱ። ከባለፈው ዓመት ጀምሮ የጤና፣ የትምህርት እና የግብር አሰባሰብ ሥራዎች በየአካባቢያቸው እየተከናወኑ አለመሆናቸውንም ለዶቼ ቬለ ተናግረዋል።",
  "የሕዝብ ተወካዮች ምክር ቤት አባል እና የቋሚ ኮሚቴ ሰብሳቢ የነበሩት አቶ ክርስቲያን ታደለ እና የአማራ ክልል ምክር ቤት አባል የሆኑት አቶ ዮሐንስ ቧያለው ከቃሊቲ ወደ ቂሊንጦ ማረሚያ ቤት መዛወራቸውን ጠበቃቸው ተናገሩ።",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡"
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)

(5, 512)
tensor([[ 1.0000, -0.0546,  0.0800, -0.0076,  0.6473],
        [-0.0546,  1.0000,  0.4848,  0.2544,  0.1079],
        [ 0.0800,  0.4848,  1.0000,  0.3143,  0.2863],
        [-0.0076,  0.2544,  0.3143,  1.0000,  0.1186],
        [ 0.6473,  0.1079,  0.2863,  0.1186,  1.0000]])


In [ ]:
from google.colab import userdata

# push model to hub
# trainer.model.push_to_hub("bert-amharic-text-embedding-medium", token=userdata.get("HF_WRITE"))

model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

'https://huggingface.co/rasyosef/bert-amharic-text-embedding-medium/commit/2d4c50e9cf022398b41073abf6e1043e55a3cfd9'

In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("rasyosef/bert-amharic-text-embedding-medium")
# Run inference
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡",
  "ለኢትዮጵያ ብሔራዊ ባንክ ዋጋን የማረጋጋት ቀዳሚ ዓላማ ጋር የተጣጣሙ የገንዘብ ፖሊሲ ምክረ ሀሳቦችን እንዲሰጥ የተቋቋመው የኢትዮጵያ ብሔራዊ ባንክ የገንዘብ ፖሊሲ ኮሚቴ እስካለፈው ህዳር ወር የነበረው እአአ የ2024 የዋጋ ግሽበት በተለይምምግብ ነክ ምርቶች ላይ ከአንድ ዓመት በፊት ከነበው ጋር ሲነጻጸር መረጋጋት ማሳየቱን ጠቁሟል፡፡ ዶይቼ ቬለ ያነጋገራቸው የአዲስ አበባ ነዋሪዎች ግን በዚህ የሚስማሙ አይመስልም፡፡ ከአምና አንጻር ያልጨመረ ነገር የለም ባይ ናቸው፡፡ የኢኮኖሚ  ባለሙያም በሰጡን አስተያየት ጭማሪው በሁሉም ረገድ የተስተዋለ በመሆኑ የመንግስት ወጪን በመቀነስ ግብርናው ላይ አተኩሮ መስራት ምናልባትም የዋጋ መረጋጋቱን ሊያመጣ ይችላል ይላሉ፡፡"
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 512]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities, similarities.shape)
# [3, 3]


README.md:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

(3, 512)
tensor([[ 1.0000,  0.6473,  0.0116],
        [ 0.6473,  1.0000, -0.0354],
        [ 0.0116, -0.0354,  1.0000]]) torch.Size([3, 3])


In [ ]:
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "የዋጋ ግሽበት በባለሙያዎች እና ነዋሪዎች አተያይ",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡",
  "ለኢትዮጵያ ብሔራዊ ባንክ ዋጋን የማረጋጋት ቀዳሚ ዓላማ ጋር የተጣጣሙ የገንዘብ ፖሊሲ ምክረ ሀሳቦችን እንዲሰጥ የተቋቋመው የኢትዮጵያ ብሔራዊ ባንክ የገንዘብ ፖሊሲ ኮሚቴ እስካለፈው ህዳር ወር የነበረው እአአ የ2024 የዋጋ ግሽበት በተለይምምግብ ነክ ምርቶች ላይ ከአንድ ዓመት በፊት ከነበው ጋር ሲነጻጸር መረጋጋት ማሳየቱን ጠቁሟል፡፡ ዶይቼ ቬለ ያነጋገራቸው የአዲስ አበባ ነዋሪዎች ግን በዚህ የሚስማሙ አይመስልም፡፡ ከአምና አንጻር ያልጨመረ ነገር የለም ባይ ናቸው፡፡ የኢኮኖሚ  ባለሙያም በሰጡን አስተያየት ጭማሪው በሁሉም ረገድ የተስተዋለ በመሆኑ የመንግስት ወጪን በመቀነስ ግብርናው ላይ አተኩሮ መስራት ምናልባትም የዋጋ መረጋጋቱን ሊያመጣ ይችላል ይላሉ፡፡"
]
embeddings = model.encode(sentences)

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[ 1.0000,  0.1729,  0.6473,  0.0116],
        [ 0.1729,  1.0000,  0.2025,  0.5641],
        [ 0.6473,  0.2025,  1.0000, -0.0354],
        [ 0.0116,  0.5641, -0.0354,  1.0000]])


In [ ]:
sentences = [
  "አበበ በሶ በላ",
  "በቀለ ውሃ ጠጣ",
  "የኦፓል ማዕድን አውጪ ወጣቶች ስሞታ"
]
embeddings = model.encode(sentences)

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.4743, 0.3108],
        [0.4743, 1.0000, 0.3775],
        [0.3108, 0.3775, 1.0000]])
